# Path Following Algorithm API
## Platform
- Crazyflie

In [1]:
import numpy as np
import math
# import plotly.graph_objects as go
# from scipy import special
import MSPN_230203 as MSPN


Development Elements

1. Planar Bezier Curve / Planar PH Curve <- Hermite Interpolation
2. Space Bezier Curve
3. Vector Field-bases Guidance algorithm

In [2]:
# 2차원 경로점
Ps = [
    [0,0],
    [1,0],
]
Ts = [
    [1,-1],
    [1,1]
]
Ts = [MSPN.normalize(T) for T in Ts]
# Hermite Interpolation
h = MSPN.HermiteInterpolation(
    Ps = Ps,
    Ts = Ts,
    scaling = 0.3,
    mode = 0,
    tau = np.linspace(0,1,100)
)
# Planar PH Curve evaluation
q = MSPN.PlanarPHCurve(h)
tau = np.linspace(0,1,100)
pos = q(tau)
x = np.real(pos)
y = np.imag(pos)

Vector Field-based Guidance
- PlanarVehicle 모듈을 활용한다.
- 학생에게 시뮬레이션 하는 부분에 대한 설명을 진행한다.
- RK4 수치적분을 구현하자.
- 

In [ ]:
class RefCurve(crv.PlanarPHCurve):
    """VFG를 적용하기 위해 __call__()메서드 오버라이딩"""
    def __call__(self, tau, **kwargs):
        return super().__call__(tau, form="real")
        